# 로컬 개발에서 AWS EC2까지 Docker Compose, Git, GitHub Actions을 사용한 Python 애플리케이션 배포 및 자동 업데이트



# 1.로컬 환경에서 Docker Compose로 Python 애플리케이션 실행

## 1.1 프로젝트 디렉토리 생성

먼저, Python 애플리케이션이 포함된 디렉토리를 생성하고 해당 디렉토리로 이동합니다.
```
mkdir stock_analysis_app
cd stock_analysis_app
```



## 1.2 Python 스크립트 작성



Python 애플리케이션 파일을 작성합니다. stock_analysis_autogluon.py 파일을 생성하고 아래 코드를 추가합니다.
```
nano stock_analysis_autogluon.py
```


python 코드
```
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from autogluon.tabular import TabularPredictor
from datetime import datetime, timedelta
import numpy as np
import gradio as gr
import koreanize_matplotlib

# 1. 분석할 주식 리스트 (Apple, Microsoft, Amazon, Tesla, Nvidia, AMD)
tickers = {'AAPL': 'Apple', 'MSFT': 'Microsoft', 'AMZN': 'Amazon', 'TSLA': 'Tesla', 'NVDA': 'Nvidia', 'AMD': 'AMD'}

# 2. 날짜 설정 (2014년 1월 1일부터 전날까지)
end_date = datetime.now() - timedelta(days=1)  # 전날까지의 데이터
start_date = datetime.strptime("2014-01-01", "%Y-%m-%d")  # 2014년 1월 1일부터

# 3. 주식 데이터를 불러오는 함수
def load_stock_data(selected_ticker):
    if selected_ticker not in tickers:
        return f"잘못된 티커를 입력하셨습니다."
    
    # yfinance로 주식 데이터 불러오기
    df = yf.download(selected_ticker, start=start_date, end=end_date)
    
    if df.empty:
        return f"{tickers[selected_ticker]}에 대한 데이터가 없습니다."
    
    return df.head(), df.tail()  # 데이터를 head와 tail로 반환

# 4. 분석 및 시각화하는 함수
def analyze_stock(selected_ticker):
    df = yf.download(selected_ticker, start=start_date, end=end_date)

    # 데이터 전처리
    df.reset_index(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Day'] = (df['Date'] - df['Date'].min()).dt.days  # 날짜를 숫자로 변환

    # 50일, 200일 이동평균선 계산
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()

    # 매수 및 매도 시점 계산
    df['Signal'] = 0
    df.loc[50:, 'Signal'] = np.where(df['MA50'][50:] > df['MA200'][50:], 1, 0)
    df['Position'] = df['Signal'].diff()

    # AutoGluon을 이용한 종가 예측
    train_data = df[['Day', 'Close']].copy()
    train_data = train_data.rename(columns={'Close': 'label'})
    predictor = TabularPredictor(label='label').fit(train_data)

    best_model = predictor.model_best
    models = predictor.get_model_names()
    all_predictions = [predictor.predict(train_data.drop(columns=['label']), model=model) for model in models]
    mean_predictions = np.mean(np.array(all_predictions), axis=0)
    std_predictions = np.std(np.array(all_predictions), axis=0)

    mape = np.mean(np.abs((train_data['label'] - mean_predictions) / train_data['label'])) * 100

    # 향후 30일 예측값 계산
    future_dates = [end_date + timedelta(days=i) for i in range(1, 31)]
    future_days = [(date - df['Date'].min()).days for date in future_dates]
    future_df = pd.DataFrame({'Day': future_days})
    future_all_predictions = [predictor.predict(future_df, model=model) for model in models]
    future_mean_predictions = np.mean(np.array(future_all_predictions), axis=0)
    future_std_predictions = np.std(np.array(future_all_predictions), axis=0)

    # 시각화 함수
    def plot_graph(data_df, future_dates, future_mean_predictions, future_std_predictions, title):
        plt.figure(figsize=(14, 8))
        plt.plot(data_df['Date'], data_df['Close'], label=f'{selected_ticker} Actual Close Prices', color='#A1C6EA')  # 파스텔 블루
        plt.plot(data_df['Date'], data_df['MA50'], label='50-Day Moving Average', color='#F4B3C2', linestyle='--')  # 파스텔 핑크
        plt.plot(data_df['Date'], data_df['MA200'], label='200-Day Moving Average', color='#B3D4A7', linestyle='--')  # 파스텔 그린
        plt.plot(data_df[data_df['Position'] == 1]['Date'], data_df[data_df['Position'] == 1]['Close'], '^', markersize=10, color='red', lw=0, label='Buy Signal')
        plt.plot(data_df[data_df['Position'] == -1]['Date'], data_df[data_df['Position'] == -1]['Close'], 'v', markersize=10, color='blue', lw=0, label='Sell Signal')
        plt.plot(future_dates, future_mean_predictions, label='Future Predicted Prices', color='#B3D4A7', linestyle='--')
        plt.fill_between(future_dates, future_mean_predictions - future_std_predictions, future_mean_predictions + future_std_predictions, color='#B3D4A7', alpha=0.2)
        plt.xlabel('Date')
        plt.ylabel('Stock Price')
        plt.title(title)
        plt.legend()
        plt.tight_layout()
        
        # Gradio에서 그래프를 반환할 수 있도록 설정
        return plt.gcf()

    # 전체 기간 그래프
    total_graph = plot_graph(df, future_dates, future_mean_predictions, future_std_predictions, "전체 기간 및 예측")

    # 최근 3개월 데이터
    last_3_months = df[df['Date'] >= (end_date - timedelta(days=90))]
    three_month_graph = plot_graph(last_3_months, future_dates, future_mean_predictions, future_std_predictions, "최근 3개월 및 예측")

    # 최근 1개월 데이터
    last_1_month = df[df['Date'] >= (end_date - timedelta(days=30))]
    one_month_graph = plot_graph(last_1_month, future_dates, future_mean_predictions, future_std_predictions, "최근 1개월 및 예측")

    # MAPE 계산 결과 텍스트와 함께 반환
    return total_graph, three_month_graph, one_month_graph, f"{tickers[selected_ticker]} 분석 완료, MAPE: {mape:.2f}%"

# Gradio 이벤트 정의 수정
def stock_analysis(selected_ticker):
    total_graph, three_month_graph, one_month_graph, analysis_result = analyze_stock(selected_ticker)
    # 각각의 그래프와 텍스트를 개별적으로 반환
    return total_graph, three_month_graph, one_month_graph, analysis_result

# Gradio UI 수정
app = gr.Blocks()

with app:
    gr.Markdown("## 주식 데이터 조회 및 분석")
    
    stock_ticker_dropdown = gr.Dropdown(choices=list(tickers.keys()), label="주식을 선택하세요", value="AAPL")
    
    with gr.Row():
        stock_ticker_dropdown
    
    df_head_output = gr.Dataframe(label="Head 데이터")
    df_tail_output = gr.Dataframe(label="Tail 데이터")
    
    stock_view_button = gr.Button("주가 보기")
    
    with gr.Row():
        stock_view_button
    
    with gr.Row():
        df_head_output
        df_tail_output
    
    total_graph_output = gr.Plot(label="전체 기간 그래프")
    three_month_graph_output = gr.Plot(label="최근 3개월 그래프")
    one_month_graph_output = gr.Plot(label="최근 1개월 그래프")
    analysis_text_output = gr.Textbox(label="분석 결과")
    
    stock_analysis_button = gr.Button("주식 분석하기")
    
    with gr.Row():
        stock_analysis_button
    
    with gr.Row():
        total_graph_output
        three_month_graph_output
        one_month_graph_output
        analysis_text_output

    # Gradio 이벤트 정의
    stock_view_button.click(load_stock_data, inputs=stock_ticker_dropdown, outputs=[df_head_output, df_tail_output])
    stock_analysis_button.click(stock_analysis, inputs=stock_ticker_dropdown, outputs=[total_graph_output, three_month_graph_output, one_month_graph_output, analysis_text_output])

# Gradio 앱 실행
app.launch(inline=False, inbrowser=True, server_name="0.0.0.0")

```



## 1.3 requirements.txt 생성

애플리케이션에서 필요한 라이브러리 목록을 requirements.txt 파일로 정의합니다.
```
nano requirements.txt
```

```
yfinance==0.2.18
pandas
matplotlib
autogluon.tabular
gradio
koreanize-matplotlib
```

## 1.4 Dockerfile 생성

Dockerfile을 작성하여 Python 3.10 이미지를 기반으로 애플리케이션을 컨테이너로 실행하도록 설정합니다.
```
nano Dockerfile
```

```
# Python 3.10 slim 이미지를 사용
FROM python:3.10-slim

# 작업 디렉토리 설정
WORKDIR /app

# requirements.txt 복사 및 설치
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 애플리케이션 코드 복사
COPY . .

# 애플리케이션 실행
CMD ["python", "stock_analysis_autogluon.py"]
```

## 1.5 docker-compose.yml 생성

docker-compose.yml 파일을 생성하여 Docker Compose로 컨테이너를 관리합니다.
```
nano docker-compose.yml
```

```
version: '3'
services:
  stock-analysis:
    build: .
    ports:
      - "7860:7860"
    volumes:
      - .:/app
    container_name: stock_analysis_app
```

## 1.6 Docker Compose로 애플리케이션 실행

로컬에서 Docker Compose로 애플리케이션을 빌드하고 실행합니다.
```
docker-compose up --build
```
실행이 완료되면 브라우저에서 http://localhost:7860으로 접속하여 애플리케이션을 테스트할 수 있습니다.



# 2.GitHub에 프로젝트 업로드

## 2.1 GitHub에 새로운 리포지토리 생성

GitHub에서 새 리포지토리를 생성한 후, 로컬 프로젝트 디렉토리에서 Git 초기화 및 첫 번째 커밋을 진행합니다.
```
# Git 초기화
git init

# 모든 파일 추가
git add .

# 커밋
git commit -m "Initial commit"

# GitHub 리포지토리 연결
git remote add origin https://github.com/your_username/your_repository_name.git

# 원격 저장소로 푸시
git push -u origin master
```
이제 GitHub에 프로젝트가 업로드되었습니다.

# 3.EC2 인스턴스 설정 및 설치

## 3.1 EC2 인스턴스에 SSH로 접속

EC2 인스턴스에 SSH로 접속합니다.
```
ssh -i "your-key.pem" ec2-user@your-ec2-public-ip
```


## 3.2 Git 설치

EC2 인스턴스에 Git을 설치합니다.
```
sudo yum install -y git
```

## 3.3 Docker 설치

EC2 인스턴스에 Docker를 설치합니다.
```
# Docker 설치
sudo yum update -y
sudo yum install -y docker

# Docker 서비스 시작
sudo service docker start

# ec2-user에게 Docker 권한 부여
sudo usermod -a -G docker ec2-user

# 변경 사항을 적용하려면 로그아웃 후 다시 로그인
exit
```
다시 SSH로 접속합니다.
```
ssh -i "your-key.pem" ec2-user@your-ec2-public-ip
```
Docker가 정상적으로 설치되었는지 확인합니다.
```
docker --version
```

## 3.4 Docker Compose 설치

EC2 인스턴스에 Docker Compose를 설치합니다.
```
sudo curl -L "https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
```
실행 권한 부여
```
sudo chmod +x /usr/local/bin/docker-compose
```
Docker Compose 버전 확인
```
docker-compose --version
```


# 4.EC2 인스턴스에서 프로젝트 가져오기 및 실행

## 4.1 GitHub 리포지토리 클론

EC2 인스턴스에서 GitHub 리포지토리를 클론합니다.<br>
GitHub 리포지토리 클론
```
git clone https://github.com/your_username/your_repository_name.git
```
클론한 디렉토리로 이동
```
cd your_repository_name
```

## 4.2 Docker Compose 실행
EC2에서 Docker Compose로 애플리케이션을 빌드하고 실행합니다.<br>
Docker Compose 빌드 및 실행
```
docker-compose up --build
```
이제 EC2 인스턴스에서 애플리케이션이 실행됩니다. 인스턴스의 퍼블릭 IP로 접속하여 애플리케이션에 접근할 수 있습니다.

예: http://your-ec2-public-ip:7860

# 5.GitHub Actions을 사용한 1주일에 한 번 자동 업데이트

## 5.1 GitHub Secrets 설정
먼저, EC2 인스턴스에 대한 SSH 접속을 위해 GitHub Secrets에 SSH 비밀번호 또는 SSH 키를 설정해야 합니다.<br>
GitHub Repository Settings로 이동합니다.<br>
좌측 메뉴에서 Secrets and variables -> Actions로 이동한 후 New repository secret을 클릭합니다.<br>
아래와 같은 항목들을 추가합니다:<br>
AWS_HOST: EC2 퍼블릭 IP (예: 43.202.33.102)<br>
AWS_USER: EC2 접속 사용자 (예: ec2-user)<br>
AWS_KEY: SSH 비밀 키 (.pem 파일 내용)<br>


## 5.2 GitHub Actions 설정

프로젝트 루트 디렉토리에 .github/workflows 디렉토리를 생성하고, 그 안에 deploy.yml 파일을 만듭니다.
```
mkdir -p .github/workflows
nano .github/workflows/deploy.yml
```
deploy.yml 파일에 아래 내용을 추가합니다
```
name: Deploy to EC2

on:
  schedule:
    - cron: '0 0 * * 0' # 매주 일요일 00:00에 실행

jobs:
  deploy:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout code
      uses: actions/checkout@v2

    - name: Connect to EC2 and update service
      uses: appleboy/ssh-action@v0.1.3
      with:
        host: ${{ secrets.AWS_HOST }}
        username: ${{ secrets.AWS_USER }}
        key: ${{ secrets.AWS_KEY }}
        script: |
          cd your_repository_name  # 프로젝트 디렉토리로 이동
          git pull origin master  # 최신 코드 가져오기
          docker-compose down  # 기존 컨테이너 중지
          docker-compose up --build -d  # 새로운 컨테이너 빌드 및 실행
```
이 deploy.yml 파일은 GitHub Actions가 설정된 일정(매주 일요일 00:00)에 EC2 서버에 SSH로 연결하여 최신 코드를 가져오고, Docker 컨테이너를 재빌드하여 배포하는 과정을 자동으로 수행합니다.

# 6.서비스 테스트 및 관리

## 6.1 서비스 확인

EC2에서 실행 중인 Docker 컨테이너가 잘 동작하는지 확인하려면 웹 브라우저에서 EC2 인스턴스의 퍼블릭 IP와 포트 7860으로 접속합니다.<br>

예: http://your-ec2-public-ip:7860
이 주소로 접속하여 Gradio 인터페이스가 정상적으로 작동하는지 확인할 수 있습니다.

## 6.2 서비스 중지

서비스를 중지하려면 Docker Compose를 중지합니다.
```
docker-compose down
```
이 명령어를 실행하면 실행 중인 컨테이너가 중지되고 제거됩니다.

## 6.3 백그라운드에서 실행

Docker Compose를 백그라운드에서 실행하려면 -d 옵션을 사용합니다.
```
docker-compose up -d --build
```
이 명령어를 실행하면 Docker가 백그라운드에서 애플리케이션을 실행하게 되며, EC2 인스턴스를 다시 시작하거나 연결을 끊어도 애플리케이션이 계속 작동합니다.